In [7]:

import numpy as np
import math
import pandas as pd
import matplotlib as plt

In [33]:
raw_data = pd.read_csv("C:\\Users\\coco1\\PycharmProjects\\PRHM01\\05\\ds05\\haberman.data", dtype=str,sep=",", header=None)

In [54]:
# gaussian parzen window
def gaussain_f(u) : 
    return (1 / math.sqrt(2 * math.pi)) * math.exp(- u ** 2  / 2)

def u(x, x_sample, h) : 
    return (x - x_sample).sum() / h

def k(x, ds, h) : 
    re = 0
    for x_ in ds : 
        u_ = u(x, x_, h)
        g = gaussain_f(u_)
        re += g
    return re

def p(x, ds, h, d) :
    n = len(ds)
    if n == 0 : return 0
    v = h ** d
    k_ = k(x, ds, h)
    return k_ / (v * n)

In [69]:
len_ = len(raw_data)
shu = raw_data.sample(frac=1)
cv = np.array(shu.values[:,0:-1],dtype = float)
cv_l = np.array(shu.values[:,-1],dtype = int)
train = np.array(shu.values[0: int(len_ / 2),0:-1], dtype = float)
label = np.array(shu.values[0: int(len_ / 2),-1], dtype = int)
test = np.array(shu.values[int(len_ / 2): ,0:-1], dtype = float)
test_label = np.array(shu.values[int(len_ / 2):,-1], dtype = int)

In [70]:
def classify (td, tl) :
    type1_train_data = []
    type2_train_data = []
    num_1 = 0
    num_2 = 0
    for i in range(len(tl)) :
        if tl[i] == 1 : 
            type1_train_data.append(td[i])
            num_1 += 1
        if tl[i] == 2 : 
            type2_train_data.append(td[i])
            num_2 += 1
    return type1_train_data, type2_train_data, num_1, num_2
def split_into_folds(ds, label,folds_n):
    li = []
    length = len(ds)
    num = int(length / folds_n)
    start = 0
    end = num
    for i in range(folds_n) :
        if i == folds_n - 1 : end = length - 1
        test = []
        va = []
        test_l = []
        va_l = []
        for j in range(length) :
            if end >= j >= start :
                test.append(ds[j])
                test_l.append(label[j])
            else: 
                va.append(ds[j])
                va_l.append(label[j])
        li.append([test, va, test_l, va_l])
        start += num
        end += num
    return li   
def evaluate (test, tl, h, train, trl) :
    t1,t2,n1,n2 = classify(train, trl)
    score = 0
    ind = 0
    for x in test :
        label = tl[ind]
        p_1 = p(x, t1, h, x.shape[0])
        p_2 = p(x, t2, h, x.shape[0])
        if p_1 > p_2 : perdict = 1
        else : perdict = 2
        if label == perdict : score += 1
        ind += 1
    return score
        
def get_parameter_by_cross_validate(ds, label, folds_n):
        h_list = []
        result_score = []
        li = split_into_folds(ds, label, folds_n)
        for h in range(10):
            scores = []
            h_ = 0.1 * float(h) + 0.1
            for i in range(folds_n):
                training_data_i = li[i][0]
                test_data_i = li[i][1]
                train_data_label = li[i][2]
                test_data_label = li[i][3]
                score = evaluate\
                    (test_data_i, 
                     test_data_label,
                     h_, 
                     training_data_i,
                     train_data_label)
                    #print score
                scores.append(score)
            score_mean = np.mean(scores)
            # print score_mean
            result_score.append(score_mean)
            h_list.append(h_)
        print(h_list)
        print(result_score)
        h_hat = h_list[result_score.index(max(result_score))]
        
        return h_hat

In [71]:
li = get_parameter_by_cross_validate(cv,cv_l,10)
        # [0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6, 0.7000000000000001, 0.8, 0.9, 1.0]
        # [156.1, 157.9, 157.7, 157.8, 158.0, 157.1, 155.9, 155.2, 152.6, 151.9]

[0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6, 0.7000000000000001, 0.8, 0.9, 1.0]
[161.7, 162.8, 162.9, 162.8, 163.5, 162.3, 159.3, 157.6, 158.5, 157.9]


In [75]:
score = evaluate(test, test_label, li, train, label)

ratio = score / test.shape[0]

ratio


0.5